# TP4 Machine Learning - Red Neuronal

In [66]:
import bz2
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import random
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
from tensorflow import keras
import pandas as pd
import gensim
import tensorflow as tf

In [2]:
tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Lectura de Archivo

In [4]:
reviews = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TP4/silkroad_reviews.csv')

,item_id,date_created,comment,rating
268959,a1-cocaine-0-2g-sample-free-shipping-worldwide-a1,2014-10-04,resent after initial mix up - quality seems pr...,5.0
82735,1-000-us-dollars-cash-10-100-bills,2014-04-26,If you like crisp $100 bills fast this is your...,5.0
110619,ice-uncut-2-grams,2014-05-29,second order with professionals. Good stealth ...,5.0
270458,7-grams-hydro-buds,2014-10-05,"5/5. Late review, apologies. Really good weed,...",5.0
312338,1-2-g-california-kush-high-quality-free-uk-pos...,2014-11-01,"Great buds, nice smoke. Excellent stealth ship...",5.0


Armo un dataframe para entrenar y uno para testear

In [5]:
reviews = reviews.loc[:][reviews.rating <=5]

In [6]:
limite = reviews.index.size*0.9

In [7]:
top = reviews.index > limite

In [8]:
down = reviews.index <= limite 

In [9]:
test = reviews[top]

In [10]:
train = reviews[down]

In [11]:
from sklearn.metrics import accuracy_score

## Embeding de los comentarios del dataframe 

In [14]:
train['comment'] = train['comment'].str.lower()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=8732d572f6b9d240f4746521cddc310a84e9a50f380c38c65796281b2ed0eda9
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [16]:

import wget
url = 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'
filename = wget.download(url)

In [17]:
import gzip
f_in = gzip.open('GoogleNews-vectors-negative300.bin.gz', 'rb')
f_out = open('GoogleNews-vectors-negative300.bin', 'wb')
f_out.writelines(f_in)

In [18]:

import gensim
from gensim.models import Word2Vec, KeyedVectors
from sklearn.decomposition import PCA

## Utilizacion de Word2Vec

In [19]:
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True,limit=100000)

In [23]:
emb_matrix = np.concatenate([np.zeros((1,300)), 
                             np.random.normal(size=(1,300)),
                             w2v_model.wv.vectors[:500000]], axis=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [27]:
tokenized_texts = []
for texto in tqdm(train['comment'].apply(str)):
    tokenized_texts.append(nltk.word_tokenize(texto))

100%|██████████| 282783/282783 [00:56<00:00, 4988.17it/s]


Calculo el largo medio de las palabras 

In [28]:
MAX_LEN = int(np.percentile([len(t) for t in tokenized_texts], 95))
MAX_LEN

38

## Proceso de tokenizacion de los comentarios

In [30]:
def preprocess_toks(tokenized_text):
    tokenized_text =tokenized_text.copy()
    if len(tokenized_text) > MAX_LEN:
        tokenized_text = tokenized_text[:MAX_LEN]
    for i in range(len(tokenized_text)):
        if tokenized_text[i] in w2v_model.wv.vocab and w2v_model.wv.vocab[tokenized_text[i]].index<500000:
            tokenized_text[i] = w2v_model.wv.vocab[tokenized_text[i]].index+2
        else:
            tokenized_text[i] = 1
    return tokenized_text

In [32]:
for i in tqdm(range(len(tokenized_texts)), total=len(tokenized_texts)):
    tokenized_texts[i] = preprocess_toks(tokenized_texts[i])

  0%|          | 0/282783 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
100%|██████████| 282783/282783 [00:21<00:00, 13162.54it/s]


In [35]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [36]:
tokenized_texts = pad_sequences(
    tokenized_texts, maxlen=MAX_LEN, dtype='int32', padding='post',
    truncating='post', value=0
)

In [38]:
from keras.layers import Dense, Input, GRU, Embedding
from keras.models import Model

## Red Neuronal 

In [39]:
word_indexes = Input((38,), dtype='int32')

word_emb = Embedding(100002, 300, weights=[emb_matrix], 
                     trainable=False, mask_zero=True)

emb_sequence = word_emb(word_indexes) # (22, 300)
rnn1 = GRU(50, activation='tanh')(emb_sequence) # (1,50)
dense1 = Dense(50, activation='tanh')(rnn1)
dense2 = Dense(50, activation='tanh')(dense1)
dense3 = Dense(50, activation='tanh')(dense2)

out = Dense(1, activation=None)(dense3)
model = Model(inputs=word_indexes, outputs=out)

In [40]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 38)]              0         
                                                                 
 embedding (Embedding)       (None, 38, 300)           30000600  
                                                                 
 gru (GRU)                   (None, 50)                52800     
                                                                 
 dense (Dense)               (None, 50)                2550      
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 1)                 51    

## Entrenameinto de la red neuronal

In [43]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse", metrics=["mae"])

In [56]:
model.fit(tokenized_texts, train['rating'].values, validation_split=0.1, epochs=15, batch_size=256)

Epoch 1/15
995/995 [==============================] - 153s 153ms/step - loss: 0.6022 - mae: 0.2493 - val_loss: 0.9146 - val_mae: 0.3448
Epoch 2/15
995/995 [==============================] - 165s 166ms/step - loss: 0.5805 - mae: 0.2401 - val_loss: 0.9382 - val_mae: 0.3066
Epoch 3/15
995/995 [==============================] - 137s 138ms/step - loss: 0.5640 - mae: 0.2320 - val_loss: 0.9233 - val_mae: 0.3230
Epoch 4/15
995/995 [==============================] - 130s 131ms/step - loss: 0.5466 - mae: 0.2256 - val_loss: 0.9511 - val_mae: 0.3317
Epoch 5/15
995/995 [==============================] - 132s 132ms/step - loss: 0.5351 - mae: 0.2215 - val_loss: 0.9457 - val_mae: 0.2885
Epoch 6/15
995/995 [==============================] - 131s 131ms/step - loss: 0.5219 - mae: 0.2170 - val_loss: 0.9883 - val_mae: 0.3018
Epoch 7/15
995/995 [==============================] - 129s 129ms/step - loss: 0.5078 - mae: 0.2080 - val_loss: 0.9857 - val_mae: 0.3344
Epoch 8/15
995/995 [============================

In [58]:
tokenized_texts_test = []
for texto in tqdm(test['comment'].apply(str)):
    tokenized_texts_test.append(nltk.word_tokenize(texto))

100%|██████████| 31423/31423 [00:06<00:00, 4593.32it/s]


In [60]:
for i in tqdm(range(len(tokenized_texts_test)), total=len(tokenized_texts_test)):
  tokenized_texts_test[i] = preprocess_toks(tokenized_texts_test[i])

  0%|          | 0/31423 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
100%|██████████| 31423/31423 [00:02<00:00, 10815.70it/s]


In [61]:
tokenized_texts_test = pad_sequences(
    tokenized_texts_test, maxlen=MAX_LEN, dtype='int32', padding='post',
    truncating='post', value=0
)

In [63]:
preds = model.predict(tokenized_texts_test)

In [64]:
pred_list = np.round(preds,0).flatten().tolist()

## Score de la metrica roc_auc

In [67]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test.rating.values,pred_list,multi_class='ovr')

0.7808977021252786